In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne.preprocessing import ICA, corrmap, create_ecg_epochs, create_eog_epochs

import os
from glob import glob
from sklearn.preprocessing import StandardScaler
from time import time
from sklearn.decomposition import PCA,FastICA


In [2]:
all_files_path=glob('../sd/*.edf')
print(len(all_files_path))

111


In [17]:
raw = mne.io.read_raw_edf(all_files_path[100], preload=True)

Extracting EDF parameters from e:\rahat\EEG\sd\MDD S31 TASK.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 162047  =      0.000 ...   632.996 secs...


In [18]:
# raw.load_data()
raw.info
# for i in range(len(info['chs'])):print(info['chs'][i]['ch_name'])


Measurement date,"May 20, 2013 10:58:48 GMT"
Experimenter,Unknown
Participant,Wan_Aziz_W0
Digitized points,Not available
Good channels,22 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,256.00 Hz
Highpass,0.50 Hz
Lowpass,80.00 Hz


Model test


In [19]:
lowf = 30
highf = 100
filtered_eeg = raw.filter(l_freq=lowf,h_freq=highf)


epochs = mne.make_fixed_length_epochs(filtered_eeg,duration=15,overlap=1)
data = epochs.get_data()


scaler = StandardScaler()
data = scaler.fit_transform(data.reshape(-1,data.shape[-1])).reshape(data.shape)
data = np.moveaxis(data,1,2)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 30 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 30.00
- Lower transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 26.25 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 113 samples (0.441 s)

Not setting metadata
45 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 45 events and 3840 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    0.0s finished


In [20]:
data.shape #epoch , channel, lenght

(45, 3840, 22)

In [21]:
reconstructed_model = tf.keras.models.load_model("gamma_15s_50epoch_32batch.keras")
result=reconstructed_model.predict(data)
predict_value = np.mean(result)
predict_value


1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x00000228DBC848B0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


C:\Users\User\AppData\Roaming\Python\Python310\site-packages\keras\src\saving\saving_lib.py:355: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 16 variables whereas the saved optimizer has 30 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


0.9999844

In [22]:
if predict_value < 0.5 :
    print('MDD Patient')
    print((1-predict_value)*100)
else :
    print('Healthy') 
    print((predict_value)*100)

Healthy
99.99843835830688


In [ ]:

# %%capture
# xx=[]
# for i in range(len(all_files_path)):
#     raw = mne.io.read_raw_edf(all_files_path[i], preload=True)
#     lowf = 1
#     highf = 100
#     filtered_eeg = raw.filter(l_freq=lowf,h_freq=highf)


#     epochs = mne.make_fixed_length_epochs(filtered_eeg,duration=25,overlap=1)
#     data = epochs.get_data()


#     # scaler = StandardScaler()
#     # data = scaler.fit_transform(data.reshape(-1,data.shape[-1])).reshape(data.shape)
#     if data.shape[1]!=22:
#         xx.append(i)


# for i in range(len(xx)):
#     os.remove(all_files_path[xx[i]])